<a href="https://colab.research.google.com/github/Manuel-Gomez-05/SenalesySistemas2/blob/main/parcial_2_manuel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parcial 2: Señales y Sistemas 2025-II
**Profesor:** Andrés Marino Álvarez Meza, Ph.D.

**Estudiante:** Manuel Alejandro Gómez
**Fecha:** Diciembre 2025

---

## 1. Justificación Teórica y Procedimiento

### Punto 1: Modulación de Amplitud (DSB-SC)

En este punto se simula un sistema de comunicación de Doble Banda Lateral con Portadora Suprimida (DSB-SC). El objetivo es visualizar y escuchar cómo la información se transforma en cada etapa.

**A. Modelo Matemático:**

1. **Modulación:** La señal de audio (mensaje) $m(t)$ se traslada a alta frecuencia multiplicándola por una portadora sinusoidal:

$$r(t) = m(t) \cdot \cos(2\pi f_c t)$$

Esto genera dos bandas laterales centradas en $\pm f_c$, eliminando la componente DC.

2. **Demodulación Coherente (Mezcla):** Para recuperar el mensaje, multiplicamos la señal recibida nuevamente por la misma portadora (asumiendo sincronización perfecta $\theta=0$):

$$x(t) = r(t) \cdot \cos(2\pi f_c t) = m(t) \cos^2(2\pi f_c t)$$

Aplicando identidades trigonométricas, obtenemos:

$$x(t) = \frac{1}{2}m(t) + \frac{1}{2}m(t)\cos(4\pi f_c t)$$

3.  **Filtrado (FFT):** La señal resultante tiene el audio original (banda base) y una réplica en $2f_c$ (alta frecuencia). Para aislar el mensaje:
    * Se lleva la señal al dominio de la frecuencia usando la **FFT**.
    * Se aplica una **máscara ideal** (filtro rectangular) que vuelve cero todas las frecuencias mayores a la frecuencia de corte ($f_{corte}$).
    * Se aplica la **IFFT** para recuperar la señal filtrada en el tiempo.

---

### Punto 2: Sistemas Dinámicos y Equivalencia RLC

El objetivo es relacionar un sistema mecánico (Masa-Resorte-Amortiguador) con un circuito eléctrico RLC para que tengan la misma respuesta transitoria (mismo $\zeta$ y $\omega_n$).

**A. Sistema Mecánico:**
La dinámica está regida por:
$$H_{mec}(s) = \frac{1}{ms^2 + cs + k}$$
Donde la frecuencia natural es $\omega_n = \sqrt{k/m}$ y el amortiguamiento $\zeta = \frac{c}{2\sqrt{km}}$.

**B. Equivalencia Eléctrica:**
Se modela un circuito con un Inductor ($L$) en serie con un paralelo Resistencia-Capacitor ($RC$). Igualando los coeficientes de la ecuación característica del circuito con la mecánica:

1.  Término independiente: $\frac{k}{m} = \frac{1}{LC} \implies L = \frac{m}{k \cdot C}$
2.  Término lineal: $\frac{c}{m} = \frac{1}{RC} \implies R = \frac{m}{c \cdot C}$

**C. Implementación en el Dashboard:**
Dado que hay 3 incógnitas eléctricas ($R, L, C$) y solo 2 ecuaciones mecánicas, el sistema es indeterminado.
* **Solución:** Se fija el valor del **Capacitor ($C$)** como un parámetro de referencia ajustable por el usuario (o un valor comercial estándar).
* **Cálculo Dinámico:** Al variar la masa $m$, el resorte $k$ o el amortiguador $c$, el código recalcula instantáneamente $R$ y $L$ para mantener la equivalencia exacta.
* **Lazo Cerrado:** Si se activa el control proporcional ($K_p$), la "rigidez" del sistema aumenta a $k_{eq} = k + K_p$, lo cual se refleja inmediatamente en el cálculo de los componentes eléctricos equivalentes.

In [ ]:
# Instalación de librerías y parche para YouTube
!pip install streamlit numpy scipy matplotlib pandas soundfile pydub -q
!pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz -q
!apt-get install ffmpeg -y -qq
print("✅ Librerías instaladas correctamente.")

In [ ]:
%%writefile presentacion.py
import os
if not os.path.exists("pages"):
    os.makedirs("pages")

# ==========================================
# 1. CREAR PORTADA (presentacion)
# ==========================================
import streamlit as st

st.set_page_config(page_title="Parcial 2 - SyS", page_icon="📡", layout="wide")

st.title("📡 Parcial 2: Señales y Sistemas 2025-II")
st.markdown("**Profesor:** Andrés Marino Álvarez Meza, Ph.D.")
st.markdown("### **Estudiante:** Manuel Alejandro Gómez ")
st.divider()

c1, c2 = st.columns(2)
with c1:
    st.info("### 📻 Punto 1: Modulación AM")
    st.write("Simulación DSB-SC con audio real, descarga de YouTube y filtrado vía FFT.")
with c2:
    st.warning("### 🏎️ Punto 2: Sistemas Dinámicos")
    st.write("Equivalencia Masa-Resorte vs Circuito RLC. Cálculo reactivo de R, L y C.")

st.markdown("---")
st.caption("Seleccione un punto en el menú lateral 👈 | Modo Oscuro 🌑")

# ==========================================
# 2. CREAR PUNTO 1 (pages/punto1_modulacion.py)
# ==========================================
%%writefile pages/punto1_modulacion.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq, fftshift
import yt_dlp
from pydub import AudioSegment
import os

plt.style.use('dark_background')

st.set_page_config(page_title="Punto 1", layout="wide")

@st.cache_data(show_spinner=False)
def download_audio(url, fs=44100):
    filename = "temp.wav"
    try:
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': '%(id)s.%(ext)s',
            'postprocessors': [{'key': 'FFmpegExtractAudio','preferredcodec': 'wav'}],
            'quiet': True, 'no_warnings': True, 'nocheckcertificate': True,
            'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            temp_name = f"{info['id']}.wav"

        audio = AudioSegment.from_file(temp_name)
        audio = audio.set_channels(1).set_frame_rate(fs)
        # Recorte seguro 5s
        start = 15000 if len(audio) > 20000 else 0
        audio = audio[start:start+5000]

        data = np.array(audio.get_array_of_samples()).astype(np.float32)
        if np.max(np.abs(data)) > 0: data /= np.max(np.abs(data))
        try: os.remove(temp_name)
        except: pass
        return fs, data, None
    except Exception as e:
        return None, None, str(e)

def plot_spec(t, y, fs, title, col):
    N = len(y)
    Y = fftshift(fft(y))
    f = fftshift(fftfreq(N, 1/fs))
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))
    ax1.plot(t, y, color=col, lw=0.8); ax1.set_title(f"{title} (Tiempo)"); ax1.grid(alpha=0.2)
    ax2.plot(f, np.abs(Y)/N, color='#FF00FF', lw=0.8); ax2.set_title("Espectro"); ax2.set_xlim([-fs/2, fs/2]); ax2.grid(alpha=0.2)
    st.pyplot(fig)

st.title("🎹 1. Modulación AM (DSB-SC)")
st.markdown("**Estudiante:** Manuel Alejandro Gómez")

with st.sidebar:
    st.header("🎛️ Configuración")
    src = st.radio("Fuente", ["Tono Prueba", "YouTube"])
    fc = st.slider("Portadora fc", 5000, 15000, 10000, 1000)
    cut = st.slider("Corte LPF", 1000, 6000, 4000, 500)

fs = 44100
audio = None

if src == "YouTube":
    url = st.text_input("URL", "https://www.youtube.com/watch?v=kJQP7kiw5Fk")
    if st.button("Descargar"):
        with st.spinner("Procesando..."):
            _, audio, err = download_audio(url)
            if err: st.error(err)
else:
    t = np.linspace(0, 3, 3*fs)
    audio = 0.5*np.cos(2*np.pi*440*t)*np.exp(-t) + 0.3*np.cos(2*np.pi*800*t)

if audio is not None:
    t = np.linspace(0, len(audio)/fs, len(audio))
    st.markdown("### 1. Original"); st.audio(audio, sample_rate=fs); plot_spec(t, audio, fs, "Mensaje", '#00FF00')

    st.markdown("### 2. Modulada"); carrier = np.cos(2*np.pi*fc*t); mod = audio*carrier
    st.audio(mod, sample_rate=fs); plot_spec(t, mod, fs, "Modulada", '#FFFF00')

    st.markdown("### 3. Mezcla"); mix = mod*carrier
    st.audio(mix, sample_rate=fs); plot_spec(t, mix, fs, "Mixer", '#00FFFF')

    st.markdown("### 4. Recuperada (Filtro FFT)");
    Y = fft(mix); f = fftfreq(len(mix), 1/fs); mask = np.abs(f) < cut
    rec = ifft(Y*mask).real * 2
    st.audio(rec, sample_rate=fs); plot_spec(t, rec, fs, "Final", '#FF00FF')

# ==========================================
# 3. CREAR PUNTO 2 (pages/punto2_sistemas.py)
# ==========================================
%%writefile pages/punto2_sistemas.py
import streamlit as st
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

plt.style.use('dark_background')
st.set_page_config(page_title="Punto 2", layout="wide")

st.title("🏎️ 2. Sistemas Dinámicos y RLC")
st.markdown("**Estudiante:** Manuel Alejandro Gómez")

col1, col2 = st.columns([1, 2.5])

with col1:
    st.markdown("### ⚙️ Mecánico")
    m = st.number_input("Masa (kg)", 0.1, 50.0, 1.0, 0.1)
    k = st.number_input("Resorte k (N/m)", 1.0, 500.0, 25.0, 1.0)

    cc = 2*np.sqrt(k*m)
    st.caption(f"Amort. Crítico: {cc:.2f}")
    c = st.slider("Amortiguador c", 0.0, cc*3, cc*0.3, 0.1)

    st.divider()
    st.markdown("### 🎛️ Control")
    lazo = st.radio("Lazo", ["Abierto", "Cerrado (P)"])
    Kp = 0.0
    if "Cerrado" in lazo: Kp = st.slider("Ganancia Kp", 1.0, 200.0, 50.0)

    st.divider()
    st.markdown("### 🔋 Eléctrico (Referencia)")
    C_uf = st.slider("Capacitor C (µF)", 10.0, 1000.0, 100.0, 10.0)

# CÁLCULOS REACTIVOS
k_tot = k + Kp if "Cerrado" in lazo else k
den = [m, c, k_tot]; num = [Kp] if "Cerrado" in lazo else [1]
sys = signal.TransferFunction(num, den)

wn = np.sqrt(k_tot/m); zeta = c/(2*np.sqrt(k_tot*m))

# RLC EQUIVALENTE
C_elec = C_uf * 1e-6
L_elec = m / (k_tot * C_elec)
R_elec = m / (c * C_elec) if c > 1e-5 else np.inf

# SIMULACION
t_fin = 20 if wn < 1 else 50/wn
t = np.linspace(0, t_fin, 2000)
ts_vec, ys = signal.step(sys, T=t)
fv = ys[-1]

# Métricas
bounds = np.where((ys < fv*0.98) | (ys > fv*1.02))[0]
ts_val = f"{ts_vec[bounds[-1]]:.3f} s" if len(bounds)>0 else "∞"

if zeta < 1:
    crs = np.where(ys >= fv)[0]
    tr_val = f"{ts_vec[crs[0]]:.3f} s" if len(crs)>0 else "> Sim"
else:
    i10 = np.argmax(ys>=0.1*fv); i90 = np.argmax(ys>=0.9*fv)
    tr_val = f"{ts_vec[i90]-ts_vec[i10]:.3f} s"

with col2:
    st.markdown("#### ⚡ Circuito Equivalente (Calculado)")
    c_a, c_b, c_c = st.columns(3)
    c_a.metric("Resistencia R", f"{R_elec:.2f} Ω")
    c_b.metric("Inductancia L", f"{L_elec:.4f} H")
    c_c.metric("Capacitor C", f"{C_uf:.0f} µF")

    st.divider()
    st.markdown("#### 📊 Respuesta Dinámica")
    d1, d2, d3, d4 = st.columns(4)
    d1.metric("Zeta", f"{zeta:.3f}"); d2.metric("Wn", f"{wn:.2f}")
    d3.metric("Ts (2%)", ts_val); d4.metric("Tr (Subida)", tr_val)

    tab1, tab2 = st.tabs(["Tiempo", "Polos"])
    with tab1:
        u = st.radio("Entrada", ["Escalón", "Impulso", "Rampa"], horizontal=True)
        fig, ax = plt.subplots(figsize=(10, 3.5))
        if "Escalón" in u: ax.plot(ts_vec, ys, 'cyan'); ax.axhline(fv, color='w', ls='--', alpha=0.5)
        elif "Impulso" in u: ti, yi = signal.impulse(sys, T=t); ax.plot(ti, yi, 'magenta')
        else: tr, yr, _ = signal.lsim(sys, U=t, T=t); ax.plot(tr, yr, 'yellow'); ax.plot(tr, tr, 'w', ls=':', alpha=0.3)
        ax.grid(alpha=0.15); st.pyplot(fig)
    with tab2:
        fig, ax = plt.subplots(figsize=(4,4))
        ax.scatter(np.real(sys.poles), np.imag(sys.poles), c='r', marker='x', s=100)
        ax.axhline(0, color='w', alpha=0.3); ax.axvline(0, color='w', alpha=0.3); ax.grid(alpha=0.2)
        st.pyplot(fig)

ejecutar todas las celdas , en caso de que falle la celda anterior a este mensaje , reproducir nuevamente la ultima celda para cargar el link al dashboard

In [ ]:
import subprocess
import time
import re
import os
import sys

print("--- 🚀 Iniciando Servidor Streamlit ---")

# 1. Instalar Cloudflared (si no está)
if not os.path.exists("cloudflared"):
    subprocess.run(["curl", "-L", "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64", "-o", "cloudflared"], check=True, stdout=subprocess.DEVNULL)
    subprocess.run(["chmod", "+x", "cloudflared"], check=True)

# 2. Correr Streamlit en background
# Usamos un archivo para logs de streamlit también para evitar saturar la salida
with open("streamlit.log", "w") as st_log:
    proc_streamlit = subprocess.Popen(
        ["streamlit", "run", "presentacion.py", "--server.port", "8501", "--server.address", "0.0.0.0", "--theme.base", "dark"],
        stdout=st_log, stderr=subprocess.STDOUT
    )

# 3. Correr Túnel Cloudflared
# Limpiamos el log anterior si existe
if os.path.exists("tunnel.log"):
    os.remove("tunnel.log")

with open("tunnel.log", "w") as log_file:
    proc_tunnel = subprocess.Popen(
        ["./cloudflared", "tunnel", "--url", "http://localhost:8501"],
        stdout=log_file, stderr=subprocess.STDOUT
    )

print("⏳ Estableciendo túnel seguro... (Esto puede tardar hasta 15s)")

# 4. Bucle de Espera Inteligente (Polling)
url_publica = None
intentos = 0
max_intentos = 30  # Esperar máximo 30 segundos

while intentos < max_intentos:
    try:
        if os.path.exists("tunnel.log"):
            with open("tunnel.log", "r") as f:
                log_content = f.read()
                # Buscar URL con regex
                match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', log_content)
                if match:
                    url_publica = match.group(0)
                    break
    except:
        pass

    time.sleep(1) # Esperar 1 segundo antes de reintentar
    intentos += 1
    # Barra de progreso simple
    sys.stdout.write(".")
    sys.stdout.flush()

print("\n")

if url_publica:
    print("✅  ¡CONEXIÓN EXITOSA!")
    print(f"👉  {url_publica}  👈")
    print("\n(Mantén esta celda ejecutándose para que la app no se cierre)")
else:
    print("⚠️ No se pudo obtener la URL automáticamente.")
    print("Revisa si aparece en el log crudo a continuación:")
    try:
        with open("tunnel.log", "r") as f: print(f.read()[-500:])
    except: print("Log no disponible.")

# Mantener vivo el proceso
try:
    while True: time.sleep(1)
except KeyboardInterrupt:
    print("\n🛑 Procesos detenidos.")
    proc_streamlit.terminate()
    proc_tunnel.terminate()

--- 🚀 Iniciando Servidor Streamlit ---
⏳ Estableciendo túnel seguro... (Esto puede tardar hasta 15s)
....

✅  ¡CONEXIÓN EXITOSA!
👉  https://periodic-planners-scout-letters.trycloudflare.com  👈

(Mantén esta celda ejecutándose para que la app no se cierre)
